<a href="https://colab.research.google.com/github/Stephanie9606/zeno_khc_dash_nlp/blob/main/khc_twitter/KHC_Twitter_JanAug_Key_Sent_0913.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **KHC Dashboard Pulsar Data Testing File - Sentiment + Keywords**

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data Cleaning

--- KHC Data Jan - June 2023 ---

In [ ]:
khc_df_16 = pd.read_excel('/content/drive/MyDrive/Zeno_KHC_Twitter/khc_twitter_h1.xlsx')

--- KHC Data July - Aug 2023 ---

In [ ]:
khc_df_78 = pd.read_excel('/content/drive/MyDrive/Zeno_KHC_Twitter/khc_twitter_JulyAug.xlsx')

In [ ]:
# merge two data
khc_df = pd.concat([khc_df_16, khc_df_78], axis=0)

In [ ]:
# delete web link at the end of content
khc_df['content'] = khc_df['content'].str.split('https').str[0].str.strip()

In [ ]:
# check num of rows
khc_df.shape[0]

4556

In [ ]:
khc_df_tw = khc_df

--- keep only the relevant columns

In [ ]:
columns_to_keep = [0, 1, 2, 5, 6, 9, 14, 18]
khc_df_tw = khc_df_tw.iloc[:, columns_to_keep]

In [ ]:
khc_df_tw.head()

,id,search,source,content,date,url,topics,sentiment class
0,0_104815_8_1609539758241484801,104815,Twitter,"@TerenceBegleyNJ @RadioFreeTom You mean ""Catsu...",2023-01-01 13:19:16,https://twitter.com/phillips_allen1/status/160...,"Ketchup,Tomato products,Condiments,Sauces,Toma...",neutral
1,0_104815_8_1609589331206836224,104815,Twitter,@batkitto @sizzsarz @TomHazelaar I smell a cha...,2023-01-01 16:36:15,https://twitter.com/Samos89/status/16095893312...,"Legumes,Food industry,Food and drink preparati...",positive
2,0_104815_8_1609627560098557954,104815,Twitter,Check out this listing I just added to my #Pos...,2023-01-01 19:08:10,https://twitter.com/trinketssd/status/16096275...,"Cheese,Cookbook,Food and drink,Books,Cooking,H...",neutral
3,0_104815_8_1609658180715556864,104815,Twitter,@MiracleWhip @KraftHeinzCo please remove HFCS ...,2023-01-01 21:09:50,https://twitter.com/84lbs84days/status/1609658...,NaN,negative
4,0_104815_8_1609683821137645569,104815,Twitter,2023 starting out cursed.,2023-01-01 22:51:44,https://twitter.com/rit/status/160968382113764...,NaN,negative


--- clean @name, but keep @khc related, remove emojis only content ---

In [ ]:
import re
# tagged usernames to keep
allowed_names = ['@MiracleWhip', '@KraftHeinzCo', '@CheesyVelveeta'] # self define

# function to clean the content
def clean_content(row):
    # Use regex to replace the tagged names with empty strings
    cleaned_content = re.sub(rf"(?!{'|'.join(allowed_names)})@(\w+)", "", row["content"])
    return cleaned_content

# apply to df 'content' column
khc_df_tw["content"] = khc_df_tw.apply(clean_content, axis=1)

<ipython-input-88-0fc259e6087c>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  khc_df_tw["content"] = khc_df_tw.apply(clean_content, axis=1)


In [ ]:
# rm empty content
khc_df_tw = khc_df_tw[khc_df_tw["content"] != ""]

In [ ]:
#!pip3 install emoji==1.7.0

In [ ]:
# rm content contains only emoji
import emoji
def contains_only_emojis(text):
    return all(char in emoji.UNICODE_EMOJI['en'] or char.isspace() for char in text)

# Filter and delete rows where the 'content' column contains only emojis
khc_df_tw = khc_df_tw[~khc_df_tw['content'].apply(contains_only_emojis)]

In [ ]:
# rm rows with duplicated contents
khc_df_tw = khc_df_tw[~khc_df_tw.duplicated(subset='content', keep=False)]

In [ ]:
khc_df_tw.shape[0]

3111

In [ ]:
# run pulsar query again, extract relevant content (security -> food security, too many noise) (Delete "Heinz" in the future)
keywords = ["Kraft Heinz", "KHC", "Kraft Mac and Cheese", "Heinz", "Kraft Singles", "Miracle Whip", "Heinz Ketchup", "Lunchables", "Velveeta"]
topics = ["supply chain", "supply", "food security", "supplier", "recycled", "recycle", "recycling", "single-use", "plastic", "package", "packaging", "waste", "environment", "nature", "product", "quality", "planet", "crisis", "challenge", "problem", "criticism", "reputation"]

mask = khc_df_tw['content'].str.contains('|'.join(keywords), case=False) & khc_df_tw['content'].str.contains('|'.join(topics), case=False)

khc_df_tw = khc_df_tw[mask]

In [ ]:
khc_df_tw.head()

,id,search,source,content,date,url,topics,sentiment class
0,0_104815_8_1609539758241484801,104815,Twitter,"You mean ""Catsup."" Seriously, Hunts had a br...",2023-01-01 13:19:16,https://twitter.com/phillips_allen1/status/160...,"Ketchup,Tomato products,Condiments,Sauces,Toma...",neutral
1,0_104815_8_1609589331206836224,104815,Twitter,I smell a charity stream here…Each mileston...,2023-01-01 16:36:15,https://twitter.com/Samos89/status/16095893312...,"Legumes,Food industry,Food and drink preparati...",positive
6,0_104815_8_1609705628557275136,104815,Twitter,if anyone wants to get fired up ask them abou...,2023-01-02 00:18:23,https://twitter.com/rina_hamon/status/16097056...,NaN,neutral
7,0_104815_8_1609732418335899651,104815,Twitter,"just so you know, I work in a supermarket and ...",2023-01-02 02:04:50,https://twitter.com/sekcbich/status/1609732418...,"Food companies,Production and manufacturing,Fo...",positive
8,0_104815_8_1609742063314309120,104815,Twitter,So hard to believe Velveeta Voldemort suppo...,2023-01-02 02:43:10,https://twitter.com/leftcoastbabe/status/16097...,NaN,negative


In [ ]:
# save to new excel file
khc_df_tw.to_excel('/content/drive/MyDrive/Zeno_KHC_Twitter/khc_twitter_JanAug_clean.xlsx', index=False)

# Directly read cleaned file

In [ ]:
khc_df_tw = pd.read_excel('/content/drive/MyDrive/Zeno_KHC_Twitter/khc_twitter_JanAug_clean.xlsx')

# Keywords: KeyBert

--- 3111 rows takes approximately 10 mins (keywords:5, ngram:2) ---

In [ ]:
#!pip3 install keybert
from keybert import KeyBERT

In [ ]:
# enhance performance instead of n-gram
#!pip3 install keyphrase-vectorizers
#from keyphrase_vectorizers import KeyphraseCountVectorizer

In [ ]:
# init default vectorizer
#vectorizer = KeyphraseCountVectorizer()
#print(vectorizer.get_params()) # print parameters

# specify model
kw_model = KeyBERT('all-MiniLM-L12-v2')
# specify number of keywords to extract
n_keywords = 5
ngram = 2  # specify ngram of keywords

In [ ]:
# function to extract keywords for a batch of rows
def get_keywords_batch(rows):
    contents = rows['content'].tolist()
    keywords_batch = kw_model.extract_keywords(contents, stop_words='english', highlight=False, top_n=n_keywords, keyphrase_ngram_range=(1, ngram), diversity=0.2)
    return [[keyword[0] for keyword in keywords] for keywords in keywords_batch]

In [ ]:
# split the data into batches (higher effieciency)
batch_size = 100
batches = [khc_df_tw[i:i+batch_size] for i in range(0, len(khc_df_tw), batch_size)]

In [ ]:
# extract keywords for each batch and concatenate the results
keyword_lists = []
for batch in batches:
    keyword_lists.extend(get_keywords_batch(batch))

In [ ]:
# add the keyword lists as a new column to the DataFrame
khc_df_tw['keywords'] = keyword_lists

In [ ]:
# format to better fit in Excel
khc_df_tw['keywords'] = khc_df_tw['keywords'].apply(lambda x: ', '.join(x))

In [ ]:
khc_df_tw.head()

In [ ]:
khc_df_tw.to_excel('/content/drive/MyDrive/Zeno_KHC_Twitter/khc_twitter_JanAug_key5gram2_0913.xlsx', index=False)

# Directly read file with keywords

In [ ]:
khc_df_tw = pd.read_excel('/content/drive/MyDrive/Zeno_KHC_Twitter/khc_twitter_JanAug_key5gram2_0913.xlsx')

# Sentiment: TwitterBERT

--- 3110 rows, break into 500 each, each runs around 2 mins ---

In [ ]:
#!pip3 install transformers

In [ ]:
# break into 6 group
khc_df_tw1 = khc_df_tw.iloc[:500,:]
khc_df_tw2 = khc_df_tw.iloc[500:1000,:]
khc_df_tw3 = khc_df_tw.iloc[1000:1500,:]
khc_df_tw4 = khc_df_tw.iloc[1500:2000,:]
khc_df_tw5 = khc_df_tw.iloc[2000:2500,:]
khc_df_tw6 = khc_df_tw.iloc[2500:,:]

In [ ]:
khc_df_tw6

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig
from scipy.special import softmax

# Define the model and tokenizer
MODEL = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Function to preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# Function to analyze sentiment and add results to DataFrame
def analyze_sentiment(row):
    text = row["content"]
    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    sentiment_label = config.id2label[ranking[0]]
    sentiment_score = np.round(float(scores[ranking[0]]), 4)
    return sentiment_label, sentiment_score

# Apply sentiment analysis to the DataFrame
khc_df_tw1["roberta sentiment class"], khc_df_tw1["roberta sentiment score"] = zip(*khc_df_tw1.apply(analyze_sentiment, axis=1))

<ipython-input-40-5b3ee80a7c62>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  khc_df_tw1["roberta sentiment class"], khc_df_tw1["roberta sentiment score"] = zip(*khc_df_tw1.apply(analyze_sentiment, axis=1))
<ipython-input-40-5b3ee80a7c62>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  khc_df_tw1["roberta sentiment class"], khc_df_tw1["roberta sentiment score"] = zip(*khc_df_tw1.apply(analyze_sentiment, axis=1))


In [ ]:
khc_df_tw2["roberta sentiment class"], khc_df_tw2["roberta sentiment score"] = zip(*khc_df_tw2.apply(analyze_sentiment, axis=1))

<ipython-input-41-e2845d952d8c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  khc_df_tw2["roberta sentiment class"], khc_df_tw2["roberta sentiment score"] = zip(*khc_df_tw2.apply(analyze_sentiment, axis=1))
<ipython-input-41-e2845d952d8c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  khc_df_tw2["roberta sentiment class"], khc_df_tw2["roberta sentiment score"] = zip(*khc_df_tw2.apply(analyze_sentiment, axis=1))


In [ ]:
khc_df_tw3["roberta sentiment class"], khc_df_tw3["roberta sentiment score"] = zip(*khc_df_tw3.apply(analyze_sentiment, axis=1))

<ipython-input-42-26547da2cbe2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  khc_df_tw3["roberta sentiment class"], khc_df_tw3["roberta sentiment score"] = zip(*khc_df_tw3.apply(analyze_sentiment, axis=1))
<ipython-input-42-26547da2cbe2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  khc_df_tw3["roberta sentiment class"], khc_df_tw3["roberta sentiment score"] = zip(*khc_df_tw3.apply(analyze_sentiment, axis=1))


In [ ]:
khc_df_tw4["roberta sentiment class"], khc_df_tw4["roberta sentiment score"] = zip(*khc_df_tw4.apply(analyze_sentiment, axis=1))

<ipython-input-43-9de3b711d4d3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  khc_df_tw4["roberta sentiment class"], khc_df_tw4["roberta sentiment score"] = zip(*khc_df_tw4.apply(analyze_sentiment, axis=1))
<ipython-input-43-9de3b711d4d3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  khc_df_tw4["roberta sentiment class"], khc_df_tw4["roberta sentiment score"] = zip(*khc_df_tw4.apply(analyze_sentiment, axis=1))


In [ ]:
khc_df_tw5["roberta sentiment class"], khc_df_tw5["roberta sentiment score"] = zip(*khc_df_tw5.apply(analyze_sentiment, axis=1))

<ipython-input-44-515006ec4e98>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  khc_df_tw5["roberta sentiment class"], khc_df_tw5["roberta sentiment score"] = zip(*khc_df_tw5.apply(analyze_sentiment, axis=1))
<ipython-input-44-515006ec4e98>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  khc_df_tw5["roberta sentiment class"], khc_df_tw5["roberta sentiment score"] = zip(*khc_df_tw5.apply(analyze_sentiment, axis=1))


In [ ]:
khc_df_tw6["roberta sentiment class"], khc_df_tw6["roberta sentiment score"] = zip(*khc_df_tw6.apply(analyze_sentiment, axis=1))

<ipython-input-45-3ddd860a5994>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  khc_df_tw6["roberta sentiment class"], khc_df_tw6["roberta sentiment score"] = zip(*khc_df_tw6.apply(analyze_sentiment, axis=1))
<ipython-input-45-3ddd860a5994>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  khc_df_tw6["roberta sentiment class"], khc_df_tw6["roberta sentiment score"] = zip(*khc_df_tw6.apply(analyze_sentiment, axis=1))


In [ ]:
# merge 6 data
khc_df = pd.concat([khc_df_tw1, khc_df_tw2, khc_df_tw3, khc_df_tw4, khc_df_tw5, khc_df_tw6], axis=0)

In [ ]:
khc_df.head()

,id,search,source,content,date,url,topics,sentiment class,keywords,roberta sentiment class
0,0_104815_8_1609539758241484801,104815,Twitter,"You mean ""Catsup."" Seriously, Hunts had a br...",2023-01-01 13:19:16,https://twitter.com/phillips_allen1/status/160...,"Ketchup,Tomato products,Condiments,Sauces,Toma...",neutral,"ketchup, catsup, product ketchup, catsup serio...",neutral
1,0_104815_8_1609589331206836224,104815,Twitter,I smell a charity stream here…Each mileston...,2023-01-01 16:36:15,https://twitter.com/Samos89/status/16095893312...,"Legumes,Food industry,Food and drink preparati...",positive,"heinz beans, different heinz, heinz, beans, be...",neutral
2,0_104815_8_1609705628557275136,104815,Twitter,if anyone wants to get fired up ask them abou...,2023-01-02 00:18:23,https://twitter.com/rina_hamon/status/16097056...,NaN,neutral,"tub pickling, pickling kit, heinz plastic, pla...",neutral
3,0_104815_8_1609732418335899651,104815,Twitter,"just so you know, I work in a supermarket and ...",2023-01-02 02:04:50,https://twitter.com/sekcbich/status/1609732418...,"Food companies,Production and manufacturing,Fo...",positive,"packaging heinz, supermarket plastic, heinz be...",negative
4,0_104815_8_1609742063314309120,104815,Twitter,So hard to believe Velveeta Voldemort suppo...,2023-01-02 02:43:10,https://twitter.com/leftcoastbabe/status/16097...,NaN,negative,"think moderate, kevin mccarthy, voldemort supp...",negative


In [ ]:
# check the sentiment quality
khc_df.iloc[:,[3,-6,-2]]

,content,date,keywords
0,"You mean ""Catsup."" Seriously, Hunts had a br...",2023-01-01 13:19:16,"ketchup, catsup, product ketchup, catsup serio..."
1,I smell a charity stream here…Each mileston...,2023-01-01 16:36:15,"heinz beans, different heinz, heinz, beans, be..."
2,if anyone wants to get fired up ask them abou...,2023-01-02 00:18:23,"tub pickling, pickling kit, heinz plastic, pla..."
3,"just so you know, I work in a supermarket and ...",2023-01-02 02:04:50,"packaging heinz, supermarket plastic, heinz be..."
4,So hard to believe Velveeta Voldemort suppo...,2023-01-02 02:43:10,"think moderate, kevin mccarthy, voldemort supp..."
...,...,...,...
3106,Entered into the Ambient Food category by Kraf...,2023-08-30 10:22:04,"plastic sustainability, recycled plastic, snap..."
3107,Because no one taught him how to shop for ...,2023-08-30 16:03:31,"republican expensive, shop cheaper, cheaper pr..."
3108,"Just saw some weird lunatic's posts who, while...",2023-08-31 04:15:48,"cheese, cheese really, like cheese, cheese thi..."
3109,"Boozy confections of Eggos, Arby's French-fry-...",2023-08-31 16:55:31,"vodka oreo, confections eggos, boozy confectio..."


# Difference between Pulsar and Roberta sentiment

In [ ]:
# Twitter
different_df = khc_df[khc_df['sentiment class'] != khc_df['roberta sentiment class']]
different_df = different_df.iloc[:,[3,-6,-1]]
different_df

# Export the data to an excel file

In [ ]:
# this is the one with keywords and sentiment
khc_df_JanAug = khc_df

In [ ]:
khc_df_JanAug.shape[0]

3111

In [ ]:
# drop the last column (roberta sentiment score)
last_column = khc_df_JanAug.columns[-1]

# Drop the last column
khc_df_JanAug.drop(columns=last_column, inplace=True)

In [ ]:
khc_df_JanAug

In [ ]:
# save to new excel file
khc_df_JanAug.to_excel('/content/drive/MyDrive/Zeno_KHC_Twitter/khc_twitter_JanAug_key_sent_0913.xlsx', index=False)